# Split Apply Combine

Often times, we will want to split a dataset based on a key value in a column, and do something within this group, and return this information for each group.  We will often see this referred to as the **split-apply-combine** paradigm.  With Pandas, we will often use the `groupby` method to carry out the splitting piece of this. 

**OBJECTIVES**:

- Use `groupby` to split dataset into groups, and groups within groups
- Apply different methods to `groupby` objects
- Use aggregation methods on `groupby` objects
- Write and use functions of our own on `groupby` objects with `apply`

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
df = pd.DataFrame({'street': ['a', 'a', 'b', 'b', 'a'],
                  'agent': ['one', 'one', 'two', 'one', 'two'],
                  'sq_ft': np.random.randint(1000, 4500, 5),
                  'price': np.random.randint(100000, 600000, 5)})

In [3]:
df.head()

,agent,price,sq_ft,street
0,one,550211,1484,a
1,one,291454,4417,a
2,two,290136,3924,b
3,one,177647,2634,b
4,two,510089,2171,a


### `groupby` Basics

To begin, it is important to recognize that the result of using the `groupby` method is a `groupby` object.  This is an object that has simply grouped the data according to our input.  From here, we can select columns like usual and apply basic quantitative methods.  If we wanted to know the average square footage by agent in our mini-data above, we select the column we are concerned with (`df['sq_ft']`), and group this by agent(`.groupby(df['agent'])`).  We are returned an object for which we can then apply the mean to.  Below, we save our grouped data as `grouped`, and can subsequently perform a variety of methods.

In [4]:
df['sq_ft'].groupby(df['agent'])

In [6]:
#grouped = 
df['sq_ft'].groupby(df['agent']).count()

agent
one    3
two    2
Name: sq_ft, dtype: int64

In [6]:
grouped.mean()

agent
one    2897.666667
two    3080.000000
Name: sq_ft, dtype: float64

In [7]:
grouped.count()

agent
one    3
two    2
Name: sq_ft, dtype: int64

In [8]:
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
agent,,,,,,,,
one,3.0,2897.666667,615.108392,2276.0,2593.5,2911.0,3208.5,3506.0
two,2.0,3080.000000,680.236724,2599.0,2839.5,3080.0,3320.5,3561.0


We can pass more than one value to group by, and will be returned an object that has two levels of indices.  For example, if we wanted to know:

$$
\textit{What is the average price for each agent by street?}
$$

We will select the price column, group the data by agent and street, and apply the mean method to this.  Notice that there are two levels of indices, *agent* and *street*.  We can return a dataframe that unstacks the levels with the `unstack` method.  

In [9]:
means = df['price'].groupby([df['agent'], df['street']]).mean()

In [10]:
means

agent  street
one    a         463596.5
       b         319310.0
two    a         387356.0
       b         437566.0
Name: price, dtype: float64

In [11]:
means.keys()

MultiIndex(levels=[['one', 'two'], ['a', 'b']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['agent', 'street'])

In [12]:
means.unstack()

street,a,b
agent,,
one,463596.5,319310.0
two,387356.0,437566.0


In [13]:
means[0]

463596.5

In [14]:
means[3]

437566.0

In [15]:
means.shape

(4,)

### Example I 

Using our tips dataset example, we can explore a few straightforward questions using `groupby()`.  

1. What is the average tip by gender?
2. What is the max tip on each day?
3. What is the min tip on each day by time?
4. What is the average bill by party size?

In [2]:
tips = sns.load_dataset('tips')

In [17]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [18]:
tips.groupby('sex')[['tip']].mean()

,tip
sex,
Male,3.089618
Female,2.833448


In [19]:
tips.groupby('day')[['tip']].max()

,tip
day,
Thur,6.70
Fri,4.73
Sat,10.00
Sun,6.50


In [20]:
tips.groupby(['day', 'time'])[['tip']].min()

tip
day  time        
Thur Lunch   1.25
     Dinner  3.00
Fri  Lunch   1.58
     Dinner  1.00
Sat  Lunch    NaN
     Dinner  1.00
Sun  Lunch    NaN
     Dinner  1.01

In [21]:
tips.groupby(['size', 'day'])[[ 'tip']].mean().unstack()

tip                              
day     Thur       Fri       Sat       Sun
size                                      
1     1.8300  1.920000  1.000000       NaN
2     2.4425  2.644375  2.517547  2.816923
3     2.6925  3.000000  3.797778  3.120667
4     4.2180  4.730000  4.123846  4.087778
5     5.0000       NaN  3.000000  4.046667
6     5.3000       NaN       NaN  5.000000

Average bill and tip by smoker and time.

In [22]:
tips.groupby(['smoker', 'time'])[['total_bill', 'tip']].mean()

total_bill       tip
smoker time                        
Yes    Lunch    17.399130  2.834348
       Dinner   21.859429  3.066000
No     Lunch    17.050889  2.673778
       Dinner   20.095660  3.126887

In [23]:
tips.groupby(['smoker', 'time'])[['total_bill', 'tip']].mean().unstack()

total_bill                  tip          
time        Lunch     Dinner     Lunch    Dinner
smoker                                          
Yes     17.399130  21.859429  2.834348  3.066000
No      17.050889  20.095660  2.673778  3.126887

The results of the groupby object are iterable.  For example, if we group the tips data by smoker, we get back the group and the elements of this group.  For multiple indicies of groups, we pass these as tuples.

In [24]:
for name, group in tips.groupby(['smoker']):
    print(name)
    print(group)

Yes
     total_bill   tip     sex smoker   day    time  size
56        38.01  3.00    Male    Yes   Sat  Dinner     4
58        11.24  1.76    Male    Yes   Sat  Dinner     2
60        20.29  3.21    Male    Yes   Sat  Dinner     2
61        13.81  2.00    Male    Yes   Sat  Dinner     2
62        11.02  1.98    Male    Yes   Sat  Dinner     2
63        18.29  3.76    Male    Yes   Sat  Dinner     4
67         3.07  1.00  Female    Yes   Sat  Dinner     1
69        15.01  2.09    Male    Yes   Sat  Dinner     2
72        26.86  3.14  Female    Yes   Sat  Dinner     2
73        25.28  5.00  Female    Yes   Sat  Dinner     2
76        17.92  3.08    Male    Yes   Sat  Dinner     2
80        19.44  3.00    Male    Yes  Thur   Lunch     2
83        32.68  5.00    Male    Yes  Thur   Lunch     2
90        28.97  3.00    Male    Yes   Fri  Dinner     2
92         5.75  1.00  Female    Yes   Fri  Dinner     2
93        16.32  4.30  Female    Yes   Fri  Dinner     2
95        40.17  4.73    Ma

In [25]:
for (n1, n2), group in tips.groupby(['sex', 'smoker']):
    print(n1, n2)
    print(group)

Male Yes
     total_bill    tip   sex smoker   day    time  size
56        38.01   3.00  Male    Yes   Sat  Dinner     4
58        11.24   1.76  Male    Yes   Sat  Dinner     2
60        20.29   3.21  Male    Yes   Sat  Dinner     2
61        13.81   2.00  Male    Yes   Sat  Dinner     2
62        11.02   1.98  Male    Yes   Sat  Dinner     2
63        18.29   3.76  Male    Yes   Sat  Dinner     4
69        15.01   2.09  Male    Yes   Sat  Dinner     2
76        17.92   3.08  Male    Yes   Sat  Dinner     2
80        19.44   3.00  Male    Yes  Thur   Lunch     2
83        32.68   5.00  Male    Yes  Thur   Lunch     2
90        28.97   3.00  Male    Yes   Fri  Dinner     2
95        40.17   4.73  Male    Yes   Fri  Dinner     4
96        27.28   4.00  Male    Yes   Fri  Dinner     2
97        12.03   1.50  Male    Yes   Fri  Dinner     2
98        21.01   3.00  Male    Yes   Fri  Dinner     2
105       15.36   1.64  Male    Yes   Sat  Dinner     2
106       20.49   4.06  Male    Yes   S

In [26]:
#one-liner to create dictionary of groups
pieces = dict(list(tips.groupby('sex')))

In [27]:
pieces['Male'].head()

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2


In [28]:
pieces['Female'].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
11,35.26,5.00,Female,No,Sun,Dinner,4
14,14.83,3.02,Female,No,Sun,Dinner,2
16,10.33,1.67,Female,No,Sun,Dinner,3


### Data Aggregation

Here, we are interested in using the aggreate function to apply functions that we have developed which return scalar values.  For example, we can write a function that determines the range of tips on a given day.  We are taking all of the values for tips by group, and finding a single value for this.  Similarly, we could do something like investigate the quantiles of each group.

In [3]:
grouped = tips.groupby('day')

In [7]:
def ranger(df):
    return df.max() - df.min()

In [8]:
grouped.describe()

size                                                tip            \
     count      mean       std  min  25%  50%  75%  max count      mean   
day                                                                       
Thur  62.0  2.451613  1.066285  1.0  2.0  2.0  2.0  6.0  62.0  2.771452   
Fri   19.0  2.105263  0.567131  1.0  2.0  2.0  2.0  4.0  19.0  2.734737   
Sat   87.0  2.517241  0.819275  1.0  2.0  2.0  3.0  5.0  87.0  2.993103   
Sun   76.0  2.842105  1.007341  2.0  2.0  2.0  4.0  6.0  76.0  3.255132   

      ...                  total_bill                                      \
      ...       75%    max      count       mean       std   min      25%   
day   ...                                                                   
Thur  ...    3.3625   6.70       62.0  17.682742  7.886170  7.51  12.4425   
Fri   ...    3.3650   4.73       19.0  17.151579  8.302660  5.75  12.0950   
Sat   ...    3.3700  10.00       87.0  20.441379  9.480419  3.07  13.9050   
Sun   ...    4.0000   6.50       76.0  21.410000  8.832122  7.25  14.9875   

                             
        50%      75%    max  
day                          
Thur  16.20  20.1550  43.11  
Fri   15.38  21.7500  40.17  
Sat   18.24  24.7400  50.81  
Sun   19.63  25.5975  48.17  

[4 rows x 24 columns]

In [5]:
grouped.agg(ranger)

,total_bill,tip,size
day,,,
Thur,35.60,5.45,5
Fri,34.42,3.73,3
Sat,47.74,9.00,4
Sun,40.92,5.49,4


In [6]:
grouped.quantile(0.7)

0.7,size,tip,total_bill
day,,,
Thur,2.0,3.000,18.990
Fri,2.0,3.150,19.134
Sat,3.0,3.164,22.470
Sun,3.0,3.695,24.570


In [9]:
grouped.quantile(0.9)['tip']

day
Thur    4.920
Fri     4.060
Sat     4.802
Sun     5.035
Name: tip, dtype: float64

We can add a tip percentage column as follows.

In [10]:
tips['pct_tip'] = tips['tip']/tips['total_bill']

In [11]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,pct_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [12]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg('mean')

sex     smoker
Male    Yes       0.152771
        No        0.160669
Female  Yes       0.182150
        No        0.156921
Name: pct_tip, dtype: float64

In [13]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(['mean', 'std', ranger])

mean       std    ranger
sex    smoker                              
Male   Yes     0.152771  0.090588  0.674707
       No      0.160669  0.041849  0.220186
Female Yes     0.182150  0.071595  0.360233
       No      0.156921  0.036421  0.195876

In [16]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg([('Average Tipping Percentage',
                                               
                                                 
'mean'), ('Standard Deviation', 'std'), ('Range', ranger)])

Average Tipping Percentage  Standard Deviation     Range
sex    smoker                                                          
Male   Yes                       0.152771            0.090588  0.674707
       No                        0.160669            0.041849  0.220186
Female Yes                       0.182150            0.071595  0.360233
       No                        0.156921            0.036421  0.195876

In [17]:
funcs = ['mean', 'max', 'min', 'std']

In [18]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(funcs)

mean       max       min       std
sex    smoker                                        
Male   Yes     0.152771  0.710345  0.035638  0.090588
       No      0.160669  0.291990  0.071804  0.041849
Female Yes     0.182150  0.416667  0.056433  0.071595
       No      0.156921  0.252672  0.056797  0.036421

In [19]:
#overwrite results with new
#column names
func_named = [('Average', 'mean'), ('Maximum', 'max')]

In [20]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(func_named)

Average   Maximum
sex    smoker                    
Male   Yes     0.152771  0.710345
       No      0.160669  0.291990
Female Yes     0.182150  0.416667
       No      0.156921  0.252672

In [21]:
#pass different aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': [np.min, np.max], 'pct_tip': np.max})

tip         pct_tip
               amin  amax      amax
sex    smoker                      
Male   Yes     1.00  10.0  0.710345
       No      1.25   9.0  0.291990
Female Yes     1.00   6.5  0.416667
       No      1.00   5.2  0.252672

In [22]:
#pass multiple aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

tip             pct_tip
               amin      mean      amax
sex    smoker                          
Male   Yes     1.00  3.051167  0.710345
       No      1.25  3.113402  0.291990
Female Yes     1.00  2.931515  0.416667
       No      1.00  2.773519  0.252672

In [45]:
#ignore index labels
tips.groupby(['sex', 'smoker'], as_index=False).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

sex smoker   tip             pct_tip
                  amin      mean      amax
0    Male    Yes  1.00  3.051167  0.710345
1    Male     No  1.25  3.113402  0.291990
2  Female    Yes  1.00  2.931515  0.416667
3  Female     No  1.00  2.773519  0.252672

### `apply`

Beyond aggregation functions, we can apply a more general call to functions that don't necessarily return a scalar value.  For example, suppose we wanted to pass a function that will take the top 5 tip percentages.  Then, we can apply this to different groupings of the data.  Because our function takes a column argument as well as a top number, we can call these using the `apply` method also.

In [23]:
def top(df, n=5, column='pct_tip'):
    return df.sort_values(by = column)[-n:]

In [25]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   pct_tip
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

In [31]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   pct_tip
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

In [48]:
tips.groupby(['smoker', 'day']).apply(top, n=2, column = 'total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 83        32.68   5.00    Male    Yes  Thur   Lunch     2   
            197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  90        28.97   3.00    Male    Yes   Fri  Dinner     2   
            95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  102       44.30   2.50  Female    Yes   Sat  Dinner     3   
            170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  184       40.55   3.00    Male    Yes   Sun  Dinner     2   
            182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 85        34.83   5.17  Female     No  Thur   Lunch     4   
            142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  91        22.49   3.50    Male     No   Fri  Dinner     2   
            94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  59        48.27   6.73    Male     No   Sat  Dinner     4   
            212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  112       38.07   4.00    Male     No   Sun  Dinner     3   
            156       48.17   5.00    Male     No   Sun  Dinner     6   

                  pct_tip  
smoker day                 
Yes    Thur 83   0.152999  
            197  0.115982  
       Fri  90   0.103555  
            95   0.117750  
       Sat  102  0.056433  
            170  0.196812  
       Sun  184  0.073983  
            182  0.077178  
No     Thur 85   0.148435  
            142  0.121389  
       Fri  91   0.155625  
            94   0.142857  
       Sat  59   0.139424  
            212  0.186220  
       Sun  112  0.105070  
            156  0.103799

In [49]:
tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,pct_tip
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990


In [50]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = [])

,count,mean,std,min,50%,max
smoker,,,,,,
Yes,93.0,0.163196,0.085119,0.035638,0.153846,0.710345
No,151.0,0.159328,0.039910,0.056797,0.155625,0.291990


In [51]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = []).unstack()

       smoker
count  Yes        93.000000
       No        151.000000
mean   Yes         0.163196
       No          0.159328
std    Yes         0.085119
       No          0.039910
min    Yes         0.035638
       No          0.056797
50%    Yes         0.153846
       No          0.155625
max    Yes         0.710345
       No          0.291990
dtype: float64

### Example for filling `na`

Suppose we have a dataset that we want to use group specific values as replacements for missing values.  In the example below, it seems we have some missing bidderrate values.  Perhaps we make the assumption that an acceptable replacement would be the average bidder rating for each auction.  To apply this, we first split the data and pass an anonymous `lambda` function that will be applied to each group.  In this case, we are filling the `na` values with the `mean`.

In [9]:
auction = pd.read_csv('data/auction/auction.csv')

In [10]:
auction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10681 entries, 0 to 10680
Data columns (total 9 columns):
auctionid       10681 non-null int64
bid             10681 non-null float64
bidtime         10681 non-null float64
bidder          10665 non-null object
bidderrate      10670 non-null float64
openbid         10681 non-null float64
price           10681 non-null float64
item            10681 non-null object
auction_type    10681 non-null object
dtypes: float64(5), int64(1), object(3)
memory usage: 751.1+ KB


In [11]:
auction.bidderrate.describe()

count    10670.000000
mean        31.936739
std        120.594074
min         -4.000000
25%          1.000000
50%          5.000000
75%         21.000000
max       3140.000000
Name: bidderrate, dtype: float64

In [12]:
auction.groupby('auctionid').mean().head()

,bid,bidtime,bidderrate,openbid,price
auctionid,,,,,
1638843936,1167.857143,4.378416,38.428571,500.0,1625.0
1638844284,362.500000,1.244913,16.500000,200.0,500.0
1638844464,554.236250,6.225163,9.125000,300.0,740.0
1638844729,284.545455,6.895609,29.636364,225.0,320.0
1638893549,144.500000,2.588555,1.600000,99.0,177.5


In [13]:
auction.head()

,auctionid,bid,bidtime,bidder,bidderrate,openbid,price,item,auction_type
0,1638893549,175.0,2.230949,schadenfreud,0.0,99.0,177.5,Cartier wristwatch,3 day auction
1,1638893549,100.0,2.600116,chuik,0.0,99.0,177.5,Cartier wristwatch,3 day auction
2,1638893549,120.0,2.600810,kiwisstuff,2.0,99.0,177.5,Cartier wristwatch,3 day auction
3,1638893549,150.0,2.601076,kiwisstuff,2.0,99.0,177.5,Cartier wristwatch,3 day auction
4,1638893549,177.5,2.909826,eli.flint@flightsafety.co,4.0,99.0,177.5,Cartier wristwatch,3 day auction


In [17]:
auction[['bidderrate', 'auctionid']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10681 entries, 0 to 10680
Data columns (total 2 columns):
bidderrate    10670 non-null float64
auctionid     10681 non-null int64
dtypes: float64(1), int64(1)
memory usage: 167.0 KB


In [18]:
auction.bidderrate = auction.bidderrate.fillna(auction.groupby(auction.auctionid).bidderrate.mean())

In [16]:
auction[['bidderrate', 'auctionid']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10681 entries, 0 to 10680
Data columns (total 2 columns):
bidderrate    10670 non-null float64
auctionid     10681 non-null int64
dtypes: float64(1), int64(1)
memory usage: 167.0 KB


**PROBLEM**

Can you adjust the above to fill in the mean bidderrate based on the specific auction?

### Further Reading

- [Pandas `groupby` documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html)
- [DataCamp article](https://www.datacamp.com/community/tutorials/pandas-split-apply-combine-groupby)